In [1]:
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
# import pyvips

import json
import time
from typing import Any
from datetime import datetime


from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable
from torch.nn import functional as F

from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR


import timm 

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform


In [ ]:
# !yes | sudo dpkg -i /kaggle/input/libvips-pyvips-installation-and-getting-started/libvips/*.deb
# !pip install /kaggle/input/libvips-pyvips-installation-and-getting-started/pyvips/pyvips-2.2.1-py2.py3-none-any.whl --no-index --find-links /kaggle/input/libvips-pyvips-installation-and-getting-started/pyvips

In [2]:
def get_loss_weight(data_path):
    num_data_samples = []
    for p in sorted(glob(os.path.join(data_path, "*"))) :
        num_data_samples.append(len(os.listdir(p)))
    return [1 - (x / sum(num_data_samples)) for x in num_data_samples]

def score(true_labels, model_preds, threshold=None) :
    model_preds = model_preds.argmax(1).detach().cpu().numpy().tolist()
    true_labels = true_labels.detach().cpu().numpy().tolist()
    return f1_score(true_labels, model_preds, average='weighted')

def save_config(config, save_path, save_name="") :
    os.makedirs(save_path, exist_ok=True)
    cfg_save_time = datetime.now().strftime('%Y_%m_%d-%H_%M_%S')
    with open(os.path.join(save_path, f"{save_name}_{cfg_save_time}.json"), 'w') as f:
        json.dump(config, f, indent="\t")

def save_img(path, img, extension=".png") :
    result, encoded_img = cv2.imencode(extension, img)
    if result:
        with open(path, mode='w+b') as f:
            encoded_img.tofile(f)

def load_img(path) :
    img_array = np.fromfile(path, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    return img

def vips_read_image(image_path, longest_edge):
    
    """
    Read image using libvips

    Parameters
    ----------
    image_path: str
        Path of the image

    Returns
    -------
    image: numpy.ndarray of shape (height, width, 3)
        Image array
    """
    
    image_thumbnail = pyvips.Image.thumbnail(image_path, longest_edge)

    return np.ndarray(
        buffer=image_thumbnail.write_to_memory(),
        dtype=np.uint8,
        shape=[image_thumbnail.height, image_thumbnail.width, image_thumbnail.bands]
    )

def read_json(path):
    with open(path, "r") as j :
        m = json.load(j)
    return m

def label_enc(label_name) : 
    return {n:idx for idx, n in enumerate(sorted(label_name))}

def label_dec(label_name) : 
    return {idx:n for idx, n in enumerate(sorted(label_name))}

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int32(W * cut_rat)
    cut_h = np.int32(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix(imgs, labels):
    lam = np.random.beta(1.0, 1.0)
    rand_index = torch.randperm(imgs.size()[0]).cuda()
    target_a = labels
    target_b = labels[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(imgs.size(), lam)
    imgs[:, :, bbx1:bbx2, bby1:bby2] = imgs[rand_index, :, bbx1:bbx2, bby1:bby2]

    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (imgs.size()[-1] * imgs.size()[-2]))

    return imgs, lam, target_a, target_b

def mixup(imgs, labels):
    lam = np.random.beta(1.0, 1.0)
    rand_index = torch.randperm(imgs.size()[0]).cuda()
    mixed_imgs = lam * imgs + (1 - lam) * imgs[rand_index, :]
    target_a, target_b = labels, labels[rand_index]

    return mixed_imgs, lam, target_a, target_b

def logging(path):
    logger = SummaryWriter(path)
    return logger

In [3]:
class Trainer() :
    def __init__(self) -> None:
        # self.train_loader = None
        # self.valid_loader = None
        # self.criterion = None
        # self.optimizer = None
        # self.model = None
        
        self.best_score = 0
        self.early_stop_cnt = 0
        
    def run(self, **cfg) :
        self.log = logging(os.path.join(cfg["log_path"], cfg["model_name"], time.strftime('%Y%m%d_%H_%M_%S', time.localtime())))
        
        start_epoch = self.train_weight_load(cfg["weight_path"]) if cfg["reuse"] else 0
            
        for e in range(start_epoch, cfg["epochs"]) :
            self.train_on_epoch(e, **cfg)
            valid_acc, valid_loss = self.valid_on_epoch(e, **cfg)
            
            self.logging({"Valid ACC" : valid_acc, "Valid Loss" : valid_loss}, e)
            self.save_checkpoint(e, valid_acc, **cfg)
            
            if self.early_stop_cnt == cfg["early_stop_patient"] :
                print("=== EARLY STOP ===")
                break
    
    def train_weight_load(self, weight_path) :
        checkpoint = torch.load(weight_path)
        self.model.load_state_dict(checkpoint['model_state_dict'])        
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        return checkpoint['epoch'] + 1
        
    def train_on_epoch(self, epoch, **cfg):
        self.model.train()
        train_acc, train_loss = [], []
        tqdm_train = tqdm(self.train_loader)
        for step, (img, label) in enumerate(tqdm_train) :
            batch_res = self.train_on_batch(img, label, **cfg)
            
            train_acc.append(batch_res["acc"])
            train_loss.append(batch_res["loss"])
            
            log = {
                "Epoch" : epoch,
                "Training Acc" : np.mean(train_acc),
                "Training Loss" : np.mean(train_loss),
            }
            tqdm_train.set_postfix(log)
            self.logging(log, step)
            
        # self.scheduler.step()
        
    def train_on_batch(self, img, label, **cfg) :
        self.optimizer.zero_grad()

        img = img.to(cfg["device"])
        label = label.to(cfg["device"])
        
        output = self.model(img)
        loss = self.criterion(output, label)
        
        loss.backward()
        self.optimizer.step()
        
        acc = score(label, output)
        
        batch_metric = {
            "acc" : acc,
            "loss" : loss.item()
        }
        return batch_metric 
    
    def valid_on_epoch(self, epoch, **cfg):
        self.model.eval()
        valid_acc, valid_loss = [], []
        tqdm_valid = tqdm(self.valid_loader)
        for step, (img, label) in enumerate(tqdm_valid) :
            batch_res = self.valid_on_batch(img, label, **cfg)
            
            valid_acc.append(batch_res["acc"])
            valid_loss.append(batch_res["loss"])
            log = {
                "Epoch" : epoch,
                "Validation Acc" : np.mean(valid_acc),
                "Validation Loss" : np.mean(valid_loss),
            }
            tqdm_valid.set_postfix(log)
            
        return np.mean(valid_acc), np.mean(valid_loss)
    
    def valid_on_batch(self, img, label, **cfg):
        img = img.to(cfg["device"])
        label = label.to(cfg["device"])
        
        output = self.model(img)
        loss = self.criterion(output, label)
        
        acc = score(label, output)
        
        batch_metric = {
            "acc" : acc,
            "loss" : loss.item()
        }
        
        return batch_metric
    
    def save_checkpoint(self, epoch, val_acc, **cfg) :
        if self.best_score < val_acc:
            self.best_score = val_acc
            torch.save({
                "epoch": epoch,
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict()
            }, os.path.join(cfg["save_path"], str(epoch) + 'E-val' + str(self.best_score) + '-' + cfg["model_name"] + '.pth'))
            self.early_stop_cnt = 0 
        else : 
            self.early_stop_cnt += 1
            
    def logging(self, log_dict, step) :
        for k, v in log_dict.items():
            if k == "Epoch" : continue
            self.log.add_scalar(k, v, step)

In [4]:
class Predictor() :
    def __init__(self) -> None:
        pass
        
    def prediction(self, **cfg) :
        self.pred_weight_load(cfg["weight_path"])
        self.model.eval()        
        model_preds = []
        with torch.no_grad() :
            for img in tqdm(self.test_loader) :
                model_preds += self.predict_on_batch(img, **cfg)
        
        self.save_to_csv(model_preds, **cfg)
        
    
    def predict_on_batch(self, img, **cfg) :
        img = img.to(cfg["device"])
        return self.model(img).argmax(1).detach().cpu().numpy().tolist()
        
    def save_to_csv(self, results, **cfg) :
        _label_dec = label_dec(cfg["label_name"])
        
        csv_file = pd.read_csv(cfg["csv_path"])
        img_name_list = [df['image_id'] for df in csv_file.iloc]
#         img_name_list = [os.path.basename(p).split(".")[0] for p in glob(os.path.join(cfg["data_path"], "*"))]
        res_label_list = [_label_dec[i] for i in results]        
        
        df = pd.DataFrame({"image_id" : img_name_list, "label":res_label_list})
        df.to_csv(os.path.join(cfg["output_path"], "submission.csv"), index=False)
    
    def pred_weight_load(self, weight_path) :
        checkpoint = torch.load(weight_path)
        self.model.load_state_dict(checkpoint['model_state_dict'])

In [5]:
class BaseModel(nn.Module) :
    def __init__(self, **cfg) -> None:
        super(BaseModel, self).__init__()
#         self.model = timm.create_model(model_name=cfg["model_name"], 
#                                        num_classes=cfg["num_classes"], 
#                                        pretrained=False,
#                                        checkpoint_path="/kaggle/input/effiv2s/efficientnet_v2s_ra2_288-a6477665.pth")
        self.model = timm.create_model(model_name=cfg["model_name"], 
                                       num_classes=cfg["num_classes"], 
                                       pretrained=True)

#         self.model = nn.Sequential(
#             self.backbone,
#             nn.Linear(1000, cfg["num_classes"])
#         )
        
    def forward(self, x) :
        return self.model(x)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, imgs, labels, transform=None, binary_mode=False):
        self.imgs = imgs
        self.labels = labels
        self.label_enc = label_enc(sorted(set(labels))) if labels != None else None
        self.transform = transform
        self.binary_mode = binary_mode
        
        self.tma_img_dicts = {}
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img_path = self.imgs[index]
        
        image = load_img(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = vips_read_image(img_path, 512)
        
        if self.transform :
            image = self.transform(image=image)['image']

        if self.labels is not None:
            if self.binary_mode :
                label = torch.tensor(self.binary_encoder(self.label_enc[self.labels[index]]), dtype=torch.long)
            else :
                label = self.label_enc[self.labels[index]]
            
            return image, label

        else:
            return image
    
    def binary_encoder(self, label):
        total_label_len = len(self.label_enc.keys())
        return [1 if i == label else 0 for i in range(total_label_len) ]
    
class DatasetCreater() :
    def __init__(self) :
        self.base_filename = ".png"
    
    def create_dataset(self, transform, **cfg) :
        img_path, label_list = self.get_data(**cfg)        
        
        if cfg["mode"] == "train" :
            save_config(transform[0].to_dict(), cfg["save_path"], save_name="train_transform")
            save_config(transform[1].to_dict(), cfg["save_path"], save_name="valid_transform")
            
            return [CustomDataset(img_path[0], label_list[0], transform=transform[0], binary_mode=cfg["binary_mode"]), 
                    CustomDataset(img_path[1], label_list[1], transform=transform[1], binary_mode=cfg["binary_mode"])]
            
        elif cfg["mode"] == 'infer' :
#             save_config(transform.to_dict(), cfg["output_path"], save_name="infer_transform")
            return CustomDataset(img_path, label_list, transform=transform)
    
    
    def create_dataloader(self, transform, **cfg) :
        ds = self.create_dataset(transform, **cfg)
        
        if isinstance(ds, list) :
            return (DataLoader(ds[0], batch_size=cfg["batch_size"], shuffle=cfg["shuffle"], num_workers=cfg["num_worker"]), 
                    DataLoader(ds[1], batch_size=cfg["batch_size"], shuffle=cfg["shuffle"], num_workers=cfg["num_worker"]))
        else :
            return DataLoader(ds, batch_size=cfg["batch_size"], shuffle=cfg["shuffle"], num_workers=cfg["num_worker"], pin_memory=True)
        
        
    def get_data(self, **cfg) :
        data_path = cfg["data_path"]
        csv_file = pd.read_csv(cfg["csv_path"])
        mode = cfg["mode"]
        
        img_path_list = []
        label_list = []
        if mode == "infer" :
            for df in csv_file.iloc :
                if os.path.exists(os.path.join(data_path, "test_images",str(df['image_id'])+".png")):
                    img_path_list.append(os.path.join(data_path, "test_images", str(df['image_id'])+".png"))

                else:
                    img_path_list.append(os.path.join(data_path, "test_thumbnails",str(df['image_id'])+"_thumbnail.png"))            
            
            return img_path_list, None
        
        else :
            for df in csv_file.iloc :
#                 if df['is_tma'] :
#                     img_path_list.append(os.path.join(data_path, "train_images", str(df['image_id'])+".png"))
#                 else :
#                     img_path_list.append(os.path.join(data_path, "train_thumbnails", str(df['image_id'])+"_thumbnail.png"))
                img_path_list.append(os.path.join(data_path, str(df['image_id'])+".png"))
                label_list.append(df['label'])
                
            train_img, valid_img, train_label, valid_label = train_test_split(img_path_list, 
                                                                              label_list, 
                                                                              test_size=0.1, 
                                                                              stratify=label_list, 
                                                                              random_state=2455)
            
            return [train_img, valid_img], [train_label, valid_label]
    

In [7]:
# class BackgroundRemove(ImageOnlyTransform):
#     def __init__(self,
#                  threshold=215, 
#                  always_apply: bool = False, p: float = 0.5):
#         super(BackgroundRemove, self).__init__(always_apply, p)
#         self.threshold = threshold
    
#     def apply(self, img, **params):
#         return self.remove_background(img)
    
#     def remove_background(self, img):
#         img[np.where((img > [self.threshold, self.threshold, self.threshold]).all(axis = 2))] = [0,0,0]
#         return img
    
#     def get_transform_init_args_names(self):
#         return ("threshold",)

In [8]:
   
class BaseMain(Trainer, Predictor, DatasetCreater) :
    def __init__(self, **cfg) -> None:
        super().__init__()
        DatasetCreater.__init__(self)
        
        self.model = BaseModel(**cfg).to(cfg["device"])
        self.optimizer = Adam(self.model.parameters(), lr=cfg["learning_rate"])
#         self.criterion = nn.CrossEntropyLoss(weight=torch.tensor(cfg['label_weight'])).to("cuda")        
        self.criterion = nn.CrossEntropyLoss().to("cuda")        
        self.scheduler = CosineAnnealingLR(self.optimizer, T_max=60, eta_min=5e-4)
        
        if cfg["mode"] == 'train' :
            self.train_loader, self.valid_loader = self.create_dataloader([self.get_transform('train', **cfg), 
                                                                           self.get_transform('valid', **cfg)], **cfg)
        elif cfg["mode"] == 'infer' :
            self.test_loader = self.create_dataloader(self.get_transform('infer', **cfg), **cfg)
            
    def train(self, **cfg) :
        self.run(**cfg)
        
    def train_on_batch(self, img, label, **cfg) :
        self.optimizer.zero_grad()

        img = img.to(cfg["device"])
        label = label.to(cfg["device"])
    
        output = self.model(img)
        loss = self.criterion(output, label)
        loss.backward()
        self.optimizer.step()
        
        acc = score(label, output)

        
        batch_metric = {
            "acc" : acc,
            "loss" : loss.item()
        }
        
        return batch_metric 

    def valid_on_batch(self, img, label, **cfg):
        img = img.to(cfg["device"])
        label = label.to(cfg["device"])
        
        if cfg["binary_mode"] :
            mixup_label = torch.argmax(label, dim=1)
        
            output = self.model(img)
            loss = self.criterion(output, label.type(torch.float32))
            
            acc = score(mixup_label, output)
        else :        
            output = self.model(img)
            loss = self.criterion(output, label)
            
            acc = score(label, output)
        batch_metric = {
            "acc" : acc,
            "loss" : loss.item()
        }
        
        return batch_metric
       
    def infer(self, **cfg) :
        self.prediction(**cfg)
    
    # def predict_on_batch(self, img, **cfg) :
        
    #     img = img.to(cfg["device"])
    #     output = self.model(img)
        
    #     # output = output * cfg["label_weight"]
    #     output = output.detach().cpu() * np.array([[cfg["label_weight"]] * output.shape[0]])[0]

    #     # binary_ = torch.softmax(output, dim=1)
    #     # binary_[binary_  > 0.9] = 1 
    #     # binary_[binary_  <= 0.9] = 0 
    #     # return output.argmax(1).detach().cpu().numpy().tolist()
        
    #     return output.argmax(1).numpy().tolist()
    
    def get_transform(self, _mode, **cfg) :
        resize = cfg["resize"]
        if _mode == 'train' :
            return A.Compose([
                A.Resize(resize, resize),
                A.OneOf([
                    A.HorizontalFlip(p=1),
                    A.VerticalFlip(p=1),
                    A.RandomRotate90(p=1)], p=1),
                
                A.RandomBrightnessContrast(
                    brightness_limit=(-0.1, 0.1),
                    contrast_limit=(-0.1, 0.1), p=0.3
                ),
                A.RandomGridShuffle((3, 3), p=0.4),
                A.Normalize(),
                ToTensorV2()
            ])
        elif _mode == 'valid' :
            return A.Compose([
                A.Resize(resize, resize),
                A.Normalize(),
                ToTensorV2()
            ])
        elif _mode == 'infer' : 
            return A.Compose([
                A.Resize(resize, resize),
                A.Normalize(),
                ToTensorV2()
            ])

In [12]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    

cfg = {
        "mode" : "train", #train, #infer
        
        "model_name" : "efficientnetv2_rw_s.ra2_in1k", #"tf_efficientnetv2_m.in21k", #"swinv2_base_window12to16_192to256_22kft1k",
        #"tf_efficientnetv2_s.in21k",#"eva_large_patch14_196.in22k_ft_in1k",#"beit_base_patch16_224.in22k_ft_in22k",
        "num_classes" : 5,
        
        "learning_rate" : 1e-4,
        "focal_alpha" : 2,
        "focal_gamma" : 2,
        "resize" : 512,
        
        "data_path" : "./data/crop_resize_rmbg", 
        "csv_path" : "./data/crop_resize_rmbg.csv",
        "epochs" : 20,
        "batch_size" : 4,
        "num_worker" : 0,
        "early_stop_patient" : 10,
        
        "binary_mode" : False,
        "reuse" : False, #True, #False
        "weight_path" : None, #"./ckpt/tf_efficientnetv2_s.in21k/rmbg_lossweight_effiv2s_512/11E-val0.5294871794871795-tf_efficientnetv2_s.in21k.pth",
        
        "save_path" : "./ckpt/tf_efficientnetv2_s.in21k/cr_rs_rb_effiv2s_512",
        "output_path" : "./output/tf_efficientnetv2_s.in21k/cr_rs_rb_effiv2s_512",
        "log_path" : "./logging",
        "device" : "cuda",
        "label_name" :["HGSC", "LGSC", "EC", "CC", "MC"],
        
#         "label_weight" : [0.4646, 0.3709, 0.2072, 0.9787, 1.0]
}        

set_seed(2455)

if cfg["mode"] == "train" :
    cfg["shuffle"] = True
elif cfg["mode"] == "infer" :
    cfg["shuffle"] = False

save_config(cfg, cfg["save_path"], save_name=cfg["mode"]+"_config")

base_main = BaseMain(**cfg)

if cfg["mode"] == "train" :
    base_main.train(**cfg)
elif cfg["mode"] == "infer" :
    base_main.infer(**cfg)

100%|██████████| 22/22 [00:01<00:00, 12.04it/s, Epoch=19, Validation Acc=0.855, Validation Loss=0.554]
